In [3]:
import sys
sys.path.append('../..')
import torch
import numpy as np
from PIL import Image
import glob
from torch import cuda
import os
import shutil
from DLBio.pytorch_helpers import get_device
from SiameseNetwork import SiameseNetwork
from SiameseNetworkDataset import SiameseNetworkDataset
import ds_ear_siamese

In [4]:
FOLDER_TEST_COMPARE = './data_compare/'
DEVICE = get_device()
net = torch.load('./models/model.pt', DEVICE)
#model = SiameseNetwork()

In [ ]:
valid_dataset = SiameseNetworkDataset(imageFolderDataset=FOLDER_TEST_COMPARE,
                                        transform=transforms.Compose([transforms.Resize((100,100)),
                                                                      transforms.ToTensor()
                                                                      ])
                                       ,should_invert=False)

test_dataloader = DataLoader(siamese_dataset,num_workers=6,batch_size=1,shuffle=True)

# is the same as:
# test_dataloader = ds_ear_siamese.get_dataloader(indices=None, batch_size=1, num_workers=6, transform_mode='siamese',
#                                                  data_path=FOLDER_TEST_COMPARE, should_invert=False)

In [ ]:
folder_dataset_test = dset.ImageFolder(root=Config.testing_dir)
siamese_dataset = SiameseNetworkDataset(imageFolderDataset=folder_dataset_test,
                                        transform=transforms.Compose([transforms.Resize((100,100)),
                                                                      transforms.ToTensor()
                                                                      ])
                                       ,should_invert=False)

test_dataloader = DataLoader(siamese_dataset,num_workers=6,batch_size=1,shuffle=True)
dataiter = iter(test_dataloader)
x0,_,_ = next(dataiter)

for i in range(10):
    _,x1,label2 = next(dataiter)
    concatenated = torch.cat((x0,x1),0)
    
    output1,output2 = net(Variable(x0).cuda(),Variable(x1).cuda())
    euclidean_distance = F.pairwise_distance(output1, output2)
    imshow(torchvision.utils.make_grid(concatenated),'Dissimilarity: {:.2f}'.format(euclidean_distance.item()))